In [92]:
import os
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from pytorch_metric_learning.losses import SelfSupervisedLoss, NTXentLoss

from proteinbind_new import EmbeddingDataset, DualEmbeddingDataset
from proteinbind_new import create_proteinbind
from transformers import pipeline

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [95]:
batch_size = 64

# GO
go_dataset = DualEmbeddingDataset('GO_files/go_AA_embeddings.pt', 'GO_files/GO_embeddings.pt', "go")

train_count = int(len(go_dataset) * 0.95)
indices = list(range(len(go_dataset)))
train_indices = indices[:train_count]
test_indices = indices[train_count:]

go_train_dataloader = DataLoader(Subset(go_dataset, train_indices), batch_size=batch_size)
go_test_dataloader = DataLoader(Subset(go_dataset, test_indices), batch_size=batch_size)

go_train_iter = iter(go_train_dataloader)
go_test_iter = iter(go_test_dataloader)

# DNA
dna_dataset = DualEmbeddingDataset('DNA_files/dna_AA_embeddings.pt', 'DNA_files/uniprot_nuc_embeddings.pt', "dna")
dna_train_dataloader = DataLoader(Subset(dna_dataset, train_indices), batch_size=batch_size)
dna_test_dataloader = DataLoader(Subset(dna_dataset, test_indices), batch_size=batch_size)

dna_train_iter = iter(dna_train_dataloader)

# Text
text_dataset = DualEmbeddingDataset('Text_files/text_AA_embeddings.pt', 'Text_files/protein_whole_text_embedding.pt', "text")
text_train_dataloader = DataLoader(Subset(text_dataset, train_indices), batch_size=batch_size)
text_test_dataloader = DataLoader(Subset(text_dataset, test_indices), batch_size=batch_size)

text_train_iter = iter(text_train_dataloader)

# MSA
#msa_dataset = DualEmbeddingDataset('MSA_files/msa_AA_embeddings.pt', 'MSA_files/MSA_embeddings.pt', "msa")
#msa_dataloader = DataLoader(dna_dataset, batch_size=batch_size)

# AA 
aa_dataset = DualEmbeddingDataset('Text_files/text_AA_embeddings.pt', 'GO_files/go_AA_embeddings.pt', "text")
aa_train_dataloader = DataLoader(Subset(aa_dataset, train_indices), batch_size=batch_size)
aa_test_dataloader = DataLoader(Subset(aa_dataset, test_indices), batch_size=batch_size)

aa_train_iter = iter(aa_train_dataloader)


model = create_proteinbind()
loss_func = SelfSupervisedLoss(NTXentLoss())

In [96]:
num_epochs = 1000
model.to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
losses = []

# Get the total number of mini-batches per epoch based on the smallest data loader
num_batches = min(len(go_train_dataloader), len(text_train_dataloader), len(dna_train_dataloader), len(aa_train_dataloader))

for epoch in tqdm(range(num_epochs)):
    for batch_idx, (go_batch, text_batch, dna_batch, aa_batch) in enumerate(zip(go_train_dataloader, text_train_dataloader, dna_train_dataloader, aa_train_dataloader)):
        batch = {
            'go': go_batch['go'].type(torch.float32),
            'text': text_batch['text'].type(torch.float32),
            'dna': dna_batch['dna'].type(torch.float32),
            'AA': aa_batch['AA'].type(torch.float32)
        }

        input_data = {
            'go': batch['go'].to(device),
            'dna': batch['dna'].to(device),
            'text': batch['text'].to(device),
            'aa': batch['AA'].to(device)
        }

        output = model(input_data)
        loss = loss_func(output["aa"], output["go"]) + loss_func(output["go"], output["aa"]) + loss_func(output["aa"], output["dna"]) + loss_func(output["dna"], output["aa"]) + loss_func(output["aa"], output["text"]) + loss_func(output["text"], output["aa"])
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if epoch != 0 and (epoch + 1) % 1 == 0:
        print(f"Epoch {epoch+1}: joint loss {sum(losses) / num_batches}")
        losses = []

  0%|          | 2/1000 [00:26<3:44:28, 13.50s/it]

Epoch 2: joint loss 49.72503261189712


  0%|          | 3/1000 [00:40<3:45:40, 13.58s/it]

Epoch 3: joint loss 24.76528094003075


  0%|          | 4/1000 [00:54<3:47:18, 13.69s/it]

...

Epoch 998: joint loss 1.4085704030371027


100%|█████████▉| 999/1000 [3:49:29<00:14, 14.14s/it]

Epoch 999: joint loss 1.4089930797681998


100%|██████████| 1000/1000 [3:49:43<00:00, 13.78s/it]

Epoch 1000: joint loss 1.4253748347492594


In [97]:
torch.save(model, 'GO_DNA_Text_model.pth')

GO 9k 1000 epochs loss 1.2401675581932068

20 steps, ~6 minutes:  GO loss 8.317567825317383